In [ ]:
import cv2
import numpy as np

def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text) + '1111111111111110'  # 结束标志

def binary_to_text(binary):
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if int(char, 2) != 0)

def hide_text(cover_path, text, stego_path):
    cover = cv2.imread(cover_path)
    binary_text = text_to_binary(text)
    data_index = 0
    data_length = len(binary_text)
    
    for row in cover:
        for pixel in row:
            for channel in range(3):
                if data_index < data_length:
                    pixel[channel] = (pixel[channel] & 0xFE) | int(binary_text[data_index])
                    data_index += 1
                else:
                    break
    cv2.imwrite(stego_path, cover)
    print("Text hidden successfully!")

def extract_text(stego_path):
    stego = cv2.imread(stego_path)
    binary_text = ''
    
    for row in stego:
        for pixel in row:
            for channel in range(3):
                binary_text += str(pixel[channel] & 1)
                if binary_text[-16:] == '1111111111111110':  # 结束标志
                    return binary_to_text(binary_text[:-16])
    return "Message not found"

# 示例用法
hide_text('./dataset/cover.jpg', 'Hello, Steganography!', './results/stego.png')
print(extract_text('./results/stego.png'))

In [ ]:
import cv2
import numpy as np
from scipy.fftpack import dct, idct

def text_to_binary(text):
    print("first 10 characters in binary t2b ", ''.join(format(ord(char), '08b') for char in text)[:10])
    return ''.join(format(ord(char), '08b') for char in text) + '1111111111111110'

def binary_to_text(binary):
    print("first 10 binary b2t ", binary[:10])
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if int(char, 2) != 0)

def dct2(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')

def idct2(block):
    return idct(idct(block.T, norm='ortho').T, norm='ortho')

def hide_text_dct(cover_path, text, stego_path):
    cover = cv2.imread(cover_path)
    binary_text = text_to_binary(text)
    print("first 10 characters in binary: ", binary_text[:10])
    data_index = 0
    data_length = len(binary_text)
    height, width, channels = cover.shape

    for channel in range(channels):
        for i in range(0, height, 8):
            for j in range(0, width, 8):
                if data_index < data_length:
                    block = cover[i:i+8, j:j+8, channel].astype(np.float32)
                    if block.shape == (8, 8):
                        dct_block = dct2(block)
                        bit = int(binary_text[data_index])
                        # print("bit: ", bit, "dct_block[7, 7]: ", dct_block[7, 7])
                        # print("original dct_block: ", dct_block)
                        pixel = dct_block[0, 0]

                        if bit == 1:
                            # make sure the last bit is 1.
                            while True:
                                dct_block[0, 0] = pixel
                                temp_original_block = idct2(dct_block)
                                temp_original_block = np.clip(temp_original_block, 0, 255)
                                temp_original_block = np.round(temp_original_block, 1)
                                temp_dct_block = dct2(temp_original_block)
                                if int(temp_dct_block[0, 0])%2 == 1:
                                    # print("temp_dct_block[7, 7]: ", temp_dct_block[7, 7])
                                    # print("int(temp_dct_block[7, 7])%2: ", int(temp_dct_block[7, 7])%2)
                                    cover[i:i+8, j:j+8, channel] = temp_original_block
                                    break
                                pixel += 1
                        else:
                            # make sure the last bit is 0.
                            while True:
                                dct_block[0, 0] = pixel
                                temp_original_block = idct2(dct_block)
                                temp_original_block = np.clip(temp_original_block, 0, 255)
                                temp_original_block = np.round(temp_original_block, 0)
                                temp_dct_block = dct2(temp_original_block)
                                if int(temp_dct_block[0, 0])%2 == 0:
                                    # print("temp_dct_block[7, 7]: ", temp_dct_block[7, 7])
                                    # print("int(temp_dct_block[7, 7])%2: ", int(temp_dct_block[7, 7])%2)
                                    cover[i:i+8, j:j+8, channel] = temp_original_block
                                    break
                                pixel += 1

                        # print("bit: ", bit, "dct_block[7, 7]: after", dct_block[7, 7])
                        data_index += 1
                        # if j<10 and i<10:
                        #     print("original dct_block: ", dct_block)
                        #     print("dct_block: ", dct_block)
                        # print("idct2(dct_block): ", idct2(dct_block))
                        # print("back dct_block: ", dct2(idct2(dct_block)))
                        # print("valid bit: ", bit)
                        # print("valid: ", int(dct2(idct2(dct_block))[7, 7])%2)
                        # cover[i:i+8, j:j+8, channel] = idct2(dct_block)
                        # print("cover[i:i+8, j:j+8, channel]: ", cover[i:i+8, j:j+8, channel])
                        # print("================》")
                        # if j>40:
                        #     return
                        
                else:
                    break
            if data_index >= data_length:
                break
        if data_index >= data_length:
            break

    cv2.imwrite(stego_path, cover)
    print("Text hidden successfully using DCT!")

def extract_text_dct(stego_path):
    stego = cv2.imread(stego_path)
    height, width, channels = stego.shape
    binary_text = ''

    for channel in range(channels):
        for i in range(0, height, 8):
            for j in range(0, width, 8):
                block = stego[i:i+8, j:j+8, channel].astype(np.float32)
                if block.shape == (8, 8):
                    dct_block = dct2(block)
                    # print("decoded dct_block: ", dct_block)
                    binary_text += str(int(dct_block[0, 0] % 2)) # extract bit from integer part.
                    print("binary_text: ", binary_text)
                    if j>100:
                        return
                    if binary_text[-16:] == '1111111111111110':
                        return binary_to_text(binary_text[:-16])
    return "Message not found"

# 示例用法
hide_text_dct('./dataset/secret2.png', '1Iello, DCT Steganography!', './results/stego_dct.png')
print("decoded")
print(extract_text_dct('./results/stego_dct.png'))

In [ ]:
import numpy as np
from scipy.fftpack import dct, idct

def dct2(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')

def idct2(block):
    return idct(idct(block.T, norm='ortho').T, norm='ortho')

# 创建一个 8x8 的随机矩阵
original_block = np.random.rand(8, 8)

# 应用 DCT
dct_block = dct2(original_block)

# 应用 IDCT
reconstructed_block = idct2(dct_block)

# 检查原始矩阵和重建矩阵是否相同
if np.allclose(original_block, reconstructed_block):
    print("dct2 和 idct2 函数工作正常！")
else:
    print("dct2 和 idct2 函数可能存在问题。")

# 打印原始矩阵和重建矩阵（可选）
#print("原始矩阵：")
#print(original_block)
#print("重建矩阵：")
#print(reconstructed_block)

In [ ]:
import cv2
import numpy as np

def dct_embed(cover_path, secret_msg, stego_path):
    # 读取载体图像并转换颜色空间
    img = cv2.imread(cover_path)
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    y_channel = ycrcb[:,:,0].astype(np.float32)

    # 将消息转换为二进制
    msg_bin = ''.join(format(ord(c), '08b') for c in secret_msg)
    msg_bin += '00000000'  # 添加终止符
    
    # 分块参数（确保处理完整块）
    block_size = 8
    rows = (y_channel.shape[0] // block_size) * block_size  # 调整高度为8的倍数
    cols = (y_channel.shape[1] // block_size) * block_size  # 调整宽度为8的倍数
    max_blocks = (rows // block_size) * (cols // block_size)
    
    if len(msg_bin) > max_blocks:
        raise ValueError(f"消息过长，最大容量为{max_blocks}比特")

    # DCT嵌入过程
    msg_index = 0
    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):
            if msg_index >= len(msg_bin):
                break
            # 处理每个完整的8x8块
            block = y_channel[i:i+block_size, j:j+block_size]
            dct_block = cv2.dct(block)
            
            # 在中频系数嵌入信息（位置(5,3)）
            coefficient = dct_block[5,3]
            embedded_bit = int(msg_bin[msg_index])
            dct_block[5,3] = coefficient - (coefficient % 1) + embedded_bit  # 保持浮点特性
            
            # 逆DCT变换
            y_channel[i:i+block_size, j:j+block_size] = cv2.idct(dct_block)
            msg_index += 1

    # 合并颜色通道并保存
    ycrcb[:,:,0] = np.clip(y_channel, 0, 255).astype(np.uint8)
    stego_img = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(stego_path, stego_img)

def dct_extract(stego_path):
    # 读取隐写图像
    img = cv2.imread(stego_path)
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    y_channel = ycrcb[:,:,0].astype(np.float32)

    # 提取二进制信息
    extracted_bin = []
    block_size = 8
    rows = (y_channel.shape[0] // block_size) * block_size  # 仅处理完整块
    cols = (y_channel.shape[1] // block_size) * block_size
    
    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):
            block = y_channel[i:i+block_size, j:j+block_size]
            dct_block = cv2.dct(block)
            
            # 从固定位置提取信息
            bit = int(round(abs(dct_block[5,3]) % 1))  # 处理浮点精度
            extracted_bin.append(str(bit))
    
    # 转换二进制为字符串
    msg = []
    for i in range(0, len(extracted_bin), 8):
        byte = extracted_bin[i:i+8]
        if len(byte) < 8:
            break
        char = chr(int(''.join(byte), 2))
        if char == '\x00':
            break
        msg.append(char)
    
    return ''.join(msg)

# 使用示例（确保载体图像尺寸为8的倍数）
cover_image = './dataset/cover.jpg'
secret_message = 'Hello, DCT Steganography!'
stego_image = './results/stego_dct.png'

dct_embed(cover_image, secret_message, stego_image)
extracted_msg = dct_extract(stego_image)
print("提取的隐藏信息:", extracted_msg)

In [ ]:
print(int(-7.62939453e-06)%2)